# Cubic Spline Benchmarking

Basics

In [109]:
import cuspatial
import cudf
import cupy as cp
import pandas as pd
import time
import itertools
import scipy.interpolate
import numpy as np
import plotly.graph_objects as go

In [96]:
t = np.array([0, 1, 2, 3, 4])
y = np.array([3, 4, 3, 2, 3])
scp_c = scipy.interpolate.CubicSpline(t, y)

In [74]:
fig = go.Figure(data=go.Scatter(x=t, y=y))
fig.show()

In [75]:
interp = np.linspace(0,4,20)
fig = go.Figure(data=go.Scatter(x=interp, y=scp_c(interp)))
fig.show()

# A GPU based-solution

One of many.

In [76]:
class cpCubicSpline:
    def _chip(self, x, y):
        '''
        Inputs:
            x: ndarray of the independent variable
            y: ndarray of dependent variable

        Requires:
            - length x = length y = n
            - n >= 2
            - x_i != x_j when i != j
            - x_{i+1} > x_i

        Outputs:
            coefs: ndarray of shape (4, n).
                Each row i represents the degree (3 - i) coefficient.
                Each column j represents the spline corresponding to interval x_j, x_{j+1}.
                For x_i < x < x_{i+1} we have:
                    f(x) ~ coefs[0, i]*x^3 + coefs[1, i]*x^2 + coefs[2, i]*x + coefs[3, i]
        Assumes:
            Natural cubic splines (S''(x_0) = S''(x_n) = 0)
        '''
        n = x.shape[0]
        h = x[1:] - x[:-1]
        b = (y[1:]-y[:-1])/h
        v = 2*(h[:-1]+h[1:])
        u = 6*(b[1:] - b[:-1])
        z = cp.zeros(n)

        M = cp.zeros((n-2, n-2))
        cp.fill_diagonal(M, v)
        cp.fill_diagonal(M[1:], h[1:-1])
        cp.fill_diagonal(M[:, 1:], h[1:-1]) # this is a no-op due to a bug in cupy
        # the following two lines corrects the above bug
        for idx, val in enumerate(h[1:-1]):
            M[idx,idx+1] = val
        z[1:-1] = cp.linalg.solve(M, u)

        a = y[:-1]
        b = b - h*(z[1:] + 2*z[:-1])/6
        c = z[:-1]/2
        d = (z[1:]-z[:-1])/(6*h)
        t = x[:-1]

        deg_3coefs = d
        deg_2coefs = (c - 3*d*t)
        deg_1coefs = (b - (2*c*t) + (3*d*t*t))
        deg_0coefs = (a - (b*t) + (c*t*t) - (d*t*t*t))

        coefs = cp.vstack((deg_3coefs, deg_2coefs, deg_1coefs, deg_0coefs))

        return coefs

    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.c = self._chip(x, y)
    
    def __call__(self, new_points):
        
        n = self.x.shape[0]
        
        #find which spline to call for each new_point
        series_x = cudf.Series(self.x).astype('float64')
        x_idx = series_x.searchsorted(new_points) - 1
        # x_idx = cp.array(np.searchsorted(cp.asnumpy(self.x), cp.asnumpy(new_points)) - 1) 
        
        #fix boundaries (0th point should be in 1st interval, nth point should be in n-1 interval)
        x_idx[x_idx==-1] = 0
        x_idx[x_idx==n-1] = n-2
        
        
        #Fix indices to be one-dimensional:
        x_idx = 4*x_idx

        #Kernel Operation: 
        eval_ker = cp.ElementwiseKernel(
            'T x, V idx, raw T coefs', 'T output',
            'output = x*x*x*coefs[idx]+x*x*coefs[idx+1]+x*coefs[idx+2]+coefs[idx+3]',
            'eval_ker')
        return eval_ker(new_points, x_idx, self.c.transpose())

In [80]:
gpu_c = cpCubicSpline(cp.asarray(t), cp.asarray(y))
fig = go.Figure(data=go.Scatter(x=interp, y=cp.asnumpy(gpu_c(cp.asarray(interp)))))
fig.show()

# Benchmarks

In [128]:
trajectory_length = [5, 15, 50, 100, 1000]
trajectory_number = [5, 15, 50, 100, 1000]
import itertools
import time
scp_speed = {}
gpu_speed = {}
for pair in itertools.product(trajectory_length, trajectory_number):
    t = np.array(np.arange(pair[0]))
    x = np.array(np.random.random(pair[0]))
    for iteration in range(pair[1]):
        old_t = time.time()
        scp_c = scipy.interpolate.CubicSpline(t, x)
        if scp_speed.get(pair) is None:
            scp_speed[pair] = [time.time() - old_t]
        else:
            scp_speed[pair].append(time.time() - old_t)
        old_t = time.time()
        gpu_t = cp.asarray(t)
        gpu_x = cp.asarray(x)
        gpu_c = cpCubicSpline(gpu_t, gpu_x)
        if gpu_speed.get(pair) is None:
            gpu_speed[pair] = [time.time() - old_t]
        else:
            gpu_speed[pair].append(time.time() - old_t)
for key in scp_speed.keys():
    scp_speed[key] = np.sum(scp_speed[key])
    gpu_speed[key] = np.sum(gpu_speed[key])
print(scp_speed)

{(5, 5): 0.0021457672119140625, (5, 15): 0.004429340362548828, (5, 50): 0.013942480087280273, (5, 100): 0.024227619171142578, (5, 1000): 0.24287962913513184, (15, 5): 0.001203298568725586, (15, 15): 0.003756284713745117, (15, 50): 0.011499166488647461, (15, 100): 0.0232088565826416, (15, 1000): 0.23789668083190918, (50, 5): 0.001214742660522461, (50, 15): 0.003641366958618164, (50, 50): 0.012668371200561523, (50, 100): 0.02436685562133789, (50, 1000): 0.2469804286956787, (100, 5): 0.0011942386627197266, (100, 15): 0.0035877227783203125, (100, 50): 0.012265443801879883, (100, 100): 0.02476644515991211, (100, 1000): 0.25179290771484375, (1000, 5): 0.0017404556274414062, (1000, 15): 0.004744529724121094, (1000, 50): 0.01510000228881836, (1000, 100): 0.030501365661621094, (1000, 1000): 0.32076573371887207}


In [129]:
print(gpu_speed)

{(5, 5): 0.010570049285888672, (5, 15): 0.02358841896057129, (5, 50): 0.07194232940673828, (5, 100): 0.12589240074157715, (5, 1000): 1.2518324851989746, (15, 5): 0.007158041000366211, (15, 15): 0.021912813186645508, (15, 50): 0.06762337684631348, (15, 100): 0.13503479957580566, (15, 1000): 1.3679695129394531, (50, 5): 0.011526346206665039, (50, 15): 0.032469987869262695, (50, 50): 0.11068415641784668, (50, 100): 0.21548891067504883, (50, 1000): 2.1864542961120605, (100, 5): 0.01354670524597168, (100, 15): 0.040506839752197266, (100, 50): 0.1422882080078125, (100, 100): 0.27449917793273926, (100, 1000): 2.8023030757904053, (1000, 5): 0.08332657814025879, (1000, 15): 0.2383890151977539, (1000, 50): 0.797619104385376, (1000, 100): 1.5785150527954102, (1000, 1000): 16.388801097869873}


# GPU Solution: Whooped by CPU

Issues:
- cupy naive memory allocation
- no memory pool
- cupy dense matrix inversion

CPU is obviously will optimized. Can we do better?

One solution: write it in C++ and Thrust.

In [154]:
ts = cudf.Series([0, 1, 2, 3, 4]).astype('float32')
xs = cudf.Series([3, 2, 3, 4, 3]).astype('float32')
ids = cudf.Series([1])
print(cuspatial.cubic_spline(ts, xs, ids))
print(cpCubicSpline(cp.asarray(ts), cp.asarray(xs)).c)

    d3   d2         d1         d0
0  0.5 -1.5   0.000000   4.000000
1 -0.5  4.5 -12.000000  12.000000
2 -0.5  4.5 -11.999999  11.999998
3  0.5 -7.5  35.999996 -51.999996
[[  0.5  -0.5  -0.5   0.5]
 [  0.    3.    3.   -6. ]
 [ -1.5  -4.5  -4.5  22.5]
 [  3.    4.    4.  -23. ]]


In [156]:
thrust_speed = {}
for pair in itertools.product(trajectory_length, trajectory_number):
    t = np.array(np.arange(pair[0]))
    x = np.array(np.random.random(pair[0]))
    for iteration in range(pair[1]):
        old_t = time.time()
        cudf_t = cudf.Series(t).astype('float32')
        cudf_x = cudf.Series(x).astype('float32')
        cudf_ids = cudf.Series([0])
        thrust_c = cuspatial.cubic_spline(cudf_t, cudf_x, cudf_ids)
        if thrust_speed.get(pair) is None:
            thrust_speed[pair] = [time.time() - old_t]
        else:
            thrust_speed[pair].append(time.time() - old_t)
for key in scp_speed.keys():
    thrust_speed[key] = np.sum(thrust_speed[key])
print(thrust_speed)

{(5, 5): 0.027508020401000977, (5, 15): 0.06812739372253418, (5, 50): 0.20779943466186523, (5, 100): 0.4096395969390869, (5, 1000): 4.34206748008728, (15, 5): 0.023364543914794922, (15, 15): 0.0715794563293457, (15, 50): 0.2350783348083496, (15, 100): 0.46721529960632324, (15, 1000): 4.368751525878906, (50, 5): 0.020088911056518555, (50, 15): 0.061742305755615234, (50, 50): 0.21335768699645996, (50, 100): 0.420330286026001, (50, 1000): 4.148272514343262, (100, 5): 0.02042698860168457, (100, 15): 0.06535005569458008, (100, 50): 0.20475101470947266, (100, 100): 0.4201383590698242, (100, 1000): 4.169664144515991, (1000, 5): 0.02098870277404785, (1000, 15): 0.06197023391723633, (1000, 50): 0.2437763214111328, (1000, 100): 0.4481492042541504, (1000, 1000): 4.137141704559326}


In [181]:
test = cpCubicSpline(cp.asarray(ts), cp.asarray(xs))
print(test(cp.asarray(cudf.Series(interp).astype('float64'))))
fig = go.Figure(data=go.Scatter(x=interp, y=cp.asnumpy(test(cp.asarray(interp)))))
fig.show()
test.c = cuspatial.cubic_spline(ts, xs, ids).to_gpu_matrix()
print(test(cp.asarray(cudf.Series(interp).astype('float32'))))
fig = go.Figure(data=go.Scatter(x=interp, y=cp.asnumpy(test(cp.asarray(cudf.Series(interp).astype('float32'))))))
fig.show()


[3.         2.68887593 2.40574428 2.17859746 2.0354279  2.00408223
 2.094766   2.28342324 2.54206153 2.84268844 3.15731156 3.45793847
 3.71657676 3.905234   3.99591777 3.9645721  3.82140254 3.59425572
 3.31112407 3.        ]


[  0.5          0.37724158   0.23815423   0.11073044  -1.4151478
 -21.406616    -9.633476     4.547016    21.358803    41.025803
   1.8967052    2.8703165    4.107522     5.6363177    7.4846916
   9.68064     12.252152    15.227218    18.633839    22.5       ]
